# Algorithms
* Sedgewick, R. & Wayne, K. **Algorithhms**. Pearson, 2011. - Java
  * https://algs4.cs.princeton.edu/
    * data: https://algs4.cs.princeton.edu/code/algs4-data.zip
  * https://github.com/kevin-wayne/algs4/

action:
- [algorithm-java.ipynb](./algorithm-java/algorithm-java.ipynb)

# 分布式ID

雪花算法: Snowflake
* [wikipedia](https://en.wikipedia.org/wiki/Snowflake_ID)

<img src="https://upload.wikimedia.org/wikipedia/commons/5/5a/Snowflake-identifier.png" width="800" />

Twitter/X - [code](https://github.com/twitter-archive/snowflake/blob/b3f6a3c6ca8e1b6847baa6ff42bf72201e2c2231/src/main/scala/com/twitter/service/snowflake/IdWorker.scala)

格式:
* 1 bit: 固定0
* 41 bits: timestamp, 从epoch的毫秒数
* 10 bits: 机器ID
  * 最多1024台机器
* 12 bits: 机器内自增序列
  * 1毫秒内的一台机器上产生4096个有序的ID

改进:
* 依赖于机器时钟: 出现时钟回拨, 直接抛出错误.
* 百度UidGenerator, 美团Leaf. - [ref](https://pdai.tech/md/algorithm/alg-domain-id-snowflake.html)


# Consistent Hashing(一致性哈希)
* https://developer.aliyun.com/article/1082388
* https://xiaolincoding.com/os/8_network_system/hash.html
* https://developer.huawei.com/consumer/cn/forum/topic/0203810951415790238

- 环形hash空间: 2^32
- 映射服务器节点
- 映射数据
- 服务器的删除与添加
- 虚拟节点

# Timing Wheel(时间轮)

ref: [时间轮（TimingWheel）高性能定时任务原理解密](https://juejin.cn/post/7083795682313633822)


时间轮 是一种 实现延迟功能（定时器） 的 巧妙算法。如果一个系统存在大量的任务调度，时间轮可以高效的利用线程资源来进行批量化调度。把大批量的调度任务全部都绑定时间轮上，通过时间轮进行所有任务的管理，触发以及运行。能够高效地管理各种延时任务，周期任务，通知任务等。

相比于 JDK 自带的 Timer、DelayQueue + ScheduledThreadPool 来说，时间轮算法是一种非常高效的调度模型。不过，时间轮调度器的时间精度可能不是很高，对于精度要求特别高的调度任务可能不太适合，因为时间轮算法的精度取决于时间段“指针”单元的最小粒度大小。比如时间轮的格子是一秒跳一次，那么调度精度小于一秒的任务就无法被时间轮所调度。

时间轮（TimingWheel）是一个 存储定时任务的环形队列，底层采用数组实现，数组中的每个元素可以存放一个定时任务列表（TimerTaskList）。TimerTaskList 是一个环形的双向链表，链表中的每一项表示的都是定时任务项（TimerTaskEntry），其中封装了真正的定时任务 TimerTask。

*基本模型构成*
- tickMs（基本时间跨度）：时间轮由多个时间格组成，每个时间格代表当前时间轮的基本时间跨度（tickMs）。
- wheelSize（时间单位个数）：时间轮的时间格个数是固定的，可用（wheelSize）来表示，那么整个时间轮的总体时间跨度（interval）可以通过公式 tickMs × wheelSize计算得出。
- currentTime（当前所处时间）：时间轮还有一个表盘指针（currentTime），用来表示时间轮当前所处的时间，currentTime 是 tickMs 的整数倍。currentTime 可以将整个时间轮划分为到期部分和未到期部分，currentTime 当前指向的时间格也属于到期部分，表示刚好到期，需要处理此时间格所对应的 TimerTaskList 的所有任务。

*问题再次深入*

如果此时有个定时为 350ms 的任务该如何处理？直接扩充 wheelSize 的大小么？
很多业务场景不乏几万甚至几十万毫秒的定时任务，这个 wheelSize 的扩充没有底线，就算将所有的定时任务的到期时间都设定一个上限，比如 100 万毫秒，那么这个 wheelSize 为 100 万毫秒的时间轮不仅占用很大的内存空间，而且效率也会拉低。所以 层级时间轮（类似十进制/二进制的计数方式）的概念应运而生，当任务的到期时间超过了当前时间轮所表示的时间范围时，就会尝试添加到上层时间轮中。

复用之前的案例，第一层的时间轮 tickMs=1ms, wheelSize=20, interval=20ms。第二层的时间轮的 tickMs 为第一层时间轮的 interval，即为 20ms。每一层时间轮的 wheelSize 是固定的，都是 20，那么第二层的时间轮的总体时间跨度 interval 为 400ms。以此类推，这个 400ms 也是第三层的 tickMs 的大小，第三层的时间轮的总体时间跨度为 8000ms。

*具体实现优化*

借助了JDK中的`DelayQueue`来协助推进时间轮。具体做法是对于每个使用到的TimerTaskList都会加入到DelayQueue中，DelayQueue会根据TimerTaskList对应的超时时间expiration来排序，最短expiration的TimerTaskList会被排在DelayQueue的队头。


# Distributed Systems

More:
* [Distributed Transaction.ipynb](./Distributed%20Transaction.ipynb)
* [Consensus.ipynb](./Consensus/Consensus.ipynb)
* [Patterns of Distributed Systems.ipynb](./Patterns%20of%20Distributed%20Systems.ipynb)

# OAuth 2.0
* https://www.ruanyifeng.com/blog/2014/05/oauth_2_0.html

RFC 6749

角色:
- Client/Third-party application(客户端)
- Resource Owner(用户)
- User Agent(用户代理): 浏览器
- HTTP Service(服务提供商)
  - Authorization Server
  - Resource Server

授权方式:
- 授权码模式(authorization code)
- 隐式/简化模式(implicit)
- 资源属主密码模式(resource owner password)
- 客户端凭证模式(client credential)

https://baijiahao.baidu.com/s?id=1705608658819242537&wfr=spider&for=pc&searchword=OAuth

OAuth的英文全称是Open Authorization，它是一种开放授权协议，*允许第三方应用程序使用资源所有者的凭据获得对资源有限访问权限*。

在OAuth2.0协议中定义了以下四个角色：

*resource owner*（资源拥有者）：能够有权授予对保护资源访问权限的实体。例如我们使用通过微信账号登录豆瓣网，而微信账号信息的实际拥有者就是微信用户，也被称为最终用户。
*resource server*（资源服务器）：承载受保护资源的服务器，能够接收使用访问令牌对受保护资源的请求并响应，它与授权服务器可以是同一服务器，也可以是不同服务器。在上述例子中该角色就是微信服务器。
*client*（客户端）：代表资源所有者及其授权发出对受保护资源请求的应用程序。在上面的例子中豆瓣网就是这样的角色。
*authorization server*（授权服务器）：认证服务器，即服务提供商专门用来处理认证授权的服务器。例如微信开放平台提供的认证服务的服务器。

OAuth2.0定义了四种授权模式，它们分别是：

*授权码模式*（authorization code）
简化模式（implicit）
密码模式（resource owner password credentials）
客户端模式（client credentials）

1. 首先微信用户点击豆瓣网微信*授权登录*按钮后，豆瓣网会将请求通过URL重定向的方式跳转至微信用户授权界面；
2. 此时微信用户实际上是在微信上进行*身份认证*，与豆瓣网并无交互了，这一点非常类似于我们使用网银支付的场景；
3. 用户使用微信客户端扫描二维码认证或者输入用户名密码后，微信会验证用户身份信息的正确性，如正确，则认为用户确认授权微信登录豆瓣网，此时会先生成一个*临时凭证*，并携带此凭证通过用户浏览器将请求*重定向*回豆瓣网在第一次重定向时携带的callBackUrl地址；
4. 之后用户浏览器会携带临时凭证code访问豆瓣网服务，豆瓣网则*通过此临时凭证再次调用微信授权接口*，获取*正式的访问凭据access_token*；
5. 在豆瓣网获取到微信授权访问凭据access_token后，此时用户的授权基本上就完成了，后续豆瓣网要做的只是*通过此token再访问微信提供的相关接口*，获取微信允许授权开发的用户信息，如头像，昵称等，并据此完成自身的用户逻辑及用户登录会话逻辑